In [1]:
import urllib
import requests
import itertools
from copy import deepcopy as mcopy
from music21 import *

from ksxml import permute, \
    nth_permutation_indices, \
    send_m21_object, \
    write_midi_file, \
    send_transport_command

In [5]:
send_transport_command('a', 'play')

<Response [200]>

In [2]:
def hashtag_music(note_names):
    midi_numbers   = [note.Note(name).pitch.midi for name in note_names]
    with_octave_up = midi_numbers + [number + 12 for number in midi_numbers]
    appendage      = with_octave_up[1:-1]
    appendage.reverse()
    forward_back   = with_octave_up + appendage
    note_list = [note.Note(n, type='16th') for n in forward_back]
    
    # When making a note from a midi number, m21 automagically inserts a 'natural' accidental
    for n in note_list:
        if n.pitch.accidental and n.pitch.accidental.name == 'natural':
            n.pitch.accidental = None
    
    s1 = stream.Part()        
    s1.append(note_list)
    return s1

In [9]:
# source material
n1 = ['C3', 'D3', 'G3', 'A3']
n1_midi = [note.Note(name).pitch.midi for name in n1]

n2 = ['C3', 'e3', 'G3', 'b3']
n2_midi = [note.Note(name).pitch.midi for name in n2]
 

n3 = ['D3', 'E3', 'A3', 'B3']
n3_midi = [note.Note(name).pitch.midi for name in n3]

cycle_me = [n1_midi, n1_midi, n2_midi, n2_midi, n3_midi, n3_midi]
cycle_me = [ns + [n + 12 for n in ns] for ns in cycle_me] # append upper octave

<Response [200]>

In [12]:
# Permute, then mirror
permutation_stream = stream.Stream()
for notes_midi, order, r in zip(itertools.cycle(cycle_me), permute(list(range(8))), range(200)):
    
    re_ordered_midi = [notes_midi[i] for i in order]

    # ramp arp back down
    reversed_midi = re_ordered_midi[1: -1]
    reversed_midi.reverse()    
    forward_back_midi = re_ordered_midi + reversed_midi
        
    # add 
    permutation_stream.append([note.Note(num, type='16th') for num in forward_back_midi])

In [38]:
# Mirror, then permute
initial_order = nth_permutation_indices(87178291190, 14)
permutation_stream = stream.Stream()
for notes_midi, order, r in zip(itertools.cycle(cycle_me), permute(initial_order), range(100)):
    
    # ramp arp back down
    reversed_midi = notes_midi[1: -1]
    reversed_midi.reverse()    
    forward_back_midi = notes_midi + reversed_midi
    
    re_ordered_midi = [forward_back_midi[i] for i in order]
        
    # add 
    permutation_stream.append([note.Note(num, type='16th') for num in re_ordered_midi])

In [24]:
# Test pushing score to browser client from a m21 stream object
stream1 = hashtag_music(n1)
send_m21_object('a', stream1)

<Response [200]>

In [39]:
# Test writing a score file
write_midi_file(permutation_stream, '/session/default.mid')

In [33]:
# Test Transport Controls
send_transport_command('a', 'play')

<Response [200]>

In [32]:
send_transport_command('a', 'stop')

<Response [200]>

In [40]:
send_transport_command('a', 'reload')

<Response [200]>